## 5. 분석
* 연간 엔터사 별 Top 앨범 출시에 따른 주가 변동률 분석하기

과정)
1. 연간 엔터사별 Top 앨범 데이터 생성 <br> (1) sql: view top album 생성하기
2. 연간 엔터사 별 Top 앨범 출시에 따른 주가 변동률 분석하기

### 1. 연간 엔터사별 top 앨범

In [2]:
import pandas as pd
import pymysql

In [3]:
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'stock_pj'
                , charset='utf8'
                ) 

In [4]:
cur = conn.cursor()
cur.execute('SHOW TABLES') # 연동 확인
cur.fetchall() # 테이블 이름 확인

(('agency_stock',),
 ('singer_agency',),
 ('top_rank_2018',),
 ('top_rank_2019',),
 ('top_rank_2020',),
 ('top_rank_2021',),
 ('top_rank_2022',),
 ('yearly_album_chart',))

- view를 이용하여 연간 엔터사별 top 앨범 df 생성하기

In [5]:
view_list = ['top_rank_2018', 'top_rank_2019', 'top_rank_2020', 'top_rank_2021', 'top_rank_2022']
year_list = [i for i in range(2018, 2023)]

yearly_top_album_tuple = ()

for i in range(len(view_list)):
    cur.execute(f'''
            select sa.agency_name, yac.year, yac.ranking, yac.singer, yac.title, yac.sales, yac.release_date
            from yearly_album_chart as yac JOIN singer_agency as sa ON yac.singer = sa.singer
            where ranking in (select * from {view_list[i]}) and year = {year_list[i]};
            ''')
    
    one_year_tuple = cur.fetchall()
    yearly_top_album_tuple += one_year_tuple

yearly_top_album_df = pd.DataFrame(yearly_top_album_tuple)
yearly_top_album_df.columns = ['agency_name', 'year', 'ranking', 'singer', 'title', 'sales', 'release_date']
yearly_top_album_df


,agency_name,year,ranking,singer,title,sales,release_date
0,SM,2018,3,EXO,DON'T MESS UP MY TEMPO - The 5th Album,1452030,2018.11.02
1,JYP,2018,10,TWICE,What is Love?,348797,2018.04.09
2,YG,2018,21,BLACKPINK,SQUARE UP,230656,2018.06.15
3,SM,2019,3,EXO,OBSESSION - The 6th Album,766294,2019.11.27
4,JYP,2019,9,슈퍼주니어 (Super Junior),Time_Slip - The 9th Album,427647,2019.10.14
5,YG,2019,17,BLACKPINK,KILL THIS LOVE,324577,2019.04.05
6,SM,2020,4,NCT,NCT RESONANCE Pt. 1 - The 2nd Album,1289594,2020.10.12
7,YG,2020,5,BLACKPINK,THE ALBUM,1244802,2020.10.02
8,JYP,2020,12,TWICE,MORE & MORE,573431,2020.06.01
9,SM,2021,2,NCT 127,Sticker - The 3rd Album,2427559,2021.09.17


In [6]:
yearly_top_album_df.shape # 3 * 5 = 15 확인 (5년간 3사의 top 앨범 정보)

(15, 7)

In [7]:
# 저장
yearly_top_album_df.to_csv('data/yearly_top_album_df.csv', encoding='utf-8', index=False)

In [9]:
conn.close()